In [10]:
# Neural style transfer from textbook

from keras.preprocessing.image import load_img, img_to_array
import os

os.chdir(path="/Users/rosenmic/Syncplicity/Professional_files/Presentations/ML_guided_CDS, incl_gen_images//")

# Path to image to transform
target_image_path = 'img/scenic_view.jpg'

# Path to image style
style_reference_image_path = 'img/starry_night_full.jpg'

#Dimensions of generated picture
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width* img_height/height)



Using TensorFlow backend.


FileNotFoundError: [Errno 2] No such file or directory: '/Users/rosenmic/Syncplicity/Professional_files/Presentations/ML_guided_CDS, incl_gen_images//'

In [2]:
# Auxillary functions
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    
    # Zero-centering by removing mean pixel value from ImageNet
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    
    # Convert BGR to RGB image 
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [3]:
# Load pretrained VGG19 network and apply to images
from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K. constant(preprocess_image(style_reference_image_path))
# Placeholder for generated image
combination_image = K.placeholder((1, img_height, img_width, 3))

# Combine images into single batch
input_tensor = K.concatenate([target_image, style_reference_image, combination_image], axis=0)
model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)
print('Model loaded.')


Model loaded.


In [4]:
# Content loss
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [5]:
# Style loss (compute Gram matrix of input matrix)
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / 4. * (channels ** 2) * (size ** 2)


In [6]:
# Total variation loss
def total_variation_loss(x):
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :] -
        x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :] -
        x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [7]:
# Define the final loss that will be minimized
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# Layer used for content loss
content_layer = 'block5_conv2'
# Layers used for style loss
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
# Weights in weighted-average of the loss components
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

# Define loss by adding all components to scalar variable
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features, combination_features)

# Add a style loss component for each target layer
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    s1 = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(style_layers)) * s1

# Add total variation loss
loss += total_variation_weight * total_variation_loss(combination_image)



In [8]:
# Set up gradient-descent process

# Get gradients of generated image with regard to loss
grads = K.gradients(loss, combination_image)[0]

# Fetch values of current loss and current gradients
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

# Class that wraps fetch_loss_and_grads to let us retrieve losses and gradients via two separate calls
class Evaluator(object):
    
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
        
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()


In [ ]:
# Run style transfer loop
from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
import time

result_prefix = './img/my_result'
iterations = 20

# Load and flatten target image (needed b/c scipy.optimize.fmin_l_bfgs_b only process flat vectors)
x = preprocess_image(target_image_path)
x = x.flatten()
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imsave(fname, img)
    print('Image saved as', fname)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))


Start of iteration 0
Current loss value: 1.24873e+32
Image saved as my_result_at_iteration_0.png
Iteration 0 completed in 494s
Start of iteration 1


/Users/rosenmic/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Current loss value: 3.55717e+31
Image saved as my_result_at_iteration_1.png
Iteration 1 completed in 494s
Start of iteration 2
Current loss value: 1.96967e+31
Image saved as my_result_at_iteration_2.png
Iteration 2 completed in 493s
Start of iteration 3
Current loss value: 1.40623e+31
Image saved as my_result_at_iteration_3.png
Iteration 3 completed in 501s
Start of iteration 4
Current loss value: 1.12556e+31
Image saved as my_result_at_iteration_4.png
Iteration 4 completed in 492s
Start of iteration 5
Current loss value: 8.94395e+30
Image saved as my_result_at_iteration_5.png
Iteration 5 completed in 492s
Start of iteration 6
Current loss value: 7.78091e+30
Image saved as my_result_at_iteration_6.png
Iteration 6 completed in 493s
Start of iteration 7
Current loss value: 6.51437e+30
Image saved as my_result_at_iteration_7.png
Iteration 7 completed in 499s
Start of iteration 8
Current loss value: 5.89903e+30
Image saved as my_result_at_iteration_8.png
Iteration 8 completed in 505s
Start